In [2]:
import csv
import re
from crypt import crypt


def create_zdrobnenina(name):
    # Pravidlá pre tvorbu zdrobnelín v slovenčine
    
    # Špeciálne prípady
    special_cases = {
        'Jozef': 'Jožko', 
        'Michal': 'Miško', 
        'Peter': 'Peťko',
        'Pavol': 'Paľko',
        'Tomáš': 'Tomáško',
        'Jakub': 'Kubko',
        'Adam': 'Adamko',
        'Daniel': 'Danko',
        'Martin': 'Martinko'
    }
    
    if name in special_cases:
        return special_cases[name]
    
    # Všeobecné pravidlá
    # Najprv skúsime odstrániť koncovky a pridať typické zdrobnelinové
    name = name.strip()
    
    # Pre ženské mená končiace na -a
    if name.endswith('a'):
        # Špeciálne prípady pre ženské mená
        if name == 'Zuzana': return 'Zuzka'
        if name == 'Katarína': return 'Katka'
        if name == 'Lucia': return 'Lucka'
        
        # Všeobecné pravidlo pre ženské mená končiace na -a
        return name[:-1] + 'ka'
    
    # Pre mužské mená končiace na spoluhlásku
    if re.search(r'[konrtlmpdvž]$', name, re.IGNORECASE):
        # Špeciálne prípady pre mužské mená
        if name == 'David': return 'Davidko'
        
        # Všeobecné pravidlo pre mužské mená
        return name + 'ko'
    
    # Pre mená končiace na -el, -er
    if name.endswith('el') or name.endswith('er'):
        return name + 'ko'
    
    # Ak žiadne pravidlo nezabralo, pridáme jednoducho -ko
    return name + 'ko'

def load_names_from_csv(filename):
    names = []
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Preskočenie hlavičky, ak existuje
        for row in reader:
            names.append(row[1])
    return names

def generate_zdrobneniny(filename):
    names = load_names_from_csv(filename)
    names_with_zdrobneniny = []

    for name in names:
        zdrobnenina = create_zdrobnenina(name)
        names_with_zdrobneniny.extend([name.lower(), zdrobnenina.lower()])
    
    return names_with_zdrobneniny

if __name__ == "__main__":
    filename = 'sk-meniny.csv'
    zdrobneniny = generate_zdrobneniny(filename)

In [4]:
from base64 import b64encode
from hashlib import md5
import itertools
from concurrent.futures import ThreadPoolExecutor
import time

def crypt(passwd, salt):
    m = md5()
    m.update(passwd.encode('utf-8'))
    m.update(salt.encode('utf-8'))
    return b64encode(m.digest()).decode('utf-8')

def load_shadow_file(filename):
    with open(filename, 'r') as file:
        return [line.strip().split(":") for line in file]
    
    
def generate_names_variants_lazy(names):
    for name in names:
        name = name.lower()
        # Yield variations with capitalized letters
        for i in range(len(name)):
            yield name[:i] + name[i].upper() + name[i+1:]
        # Yield diminutives (if the name ends in 'a')
        if name.endswith("a"):
            yield name[:-1] + "ka"

# Lazy password generator for category 2: 6-7 lowercase letters
def generate_small_letter_passwords_lazy():
    chars = "abcdefghijklmnopqrstuvwxyz"
    for length in range(6, 8):  # Lengths 6 and 7
        for password in itertools.product(chars, repeat=length):
            yield ''.join(password)

# Lazy password generator for category 3: 4-5 mixed characters
def generate_mixed_passwords_lazy():
    chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    for length in range(4, 6):  # Lengths 4 and 5
        for password in itertools.product(chars, repeat=length):
            yield ''.join(password)

def crack_single_login(login, salt, target_hash):
    start_time = time.time()
    attempts = 0
    
    password_generator = itertools.chain(
            generate_names_variants_lazy(zdrobneniny),
            # generate_small_letter_passwords_lazy(),
            # generate_mixed_passwords_lazy(),
    )
    
    for password in password_generator:
        attempts += 1
        current_hash = crypt(password, salt)
        
        if current_hash == target_hash:
            print(f"Cracked! Login: {login}, Password: {password}")
            return login, password

    return None

def crack_passwords_multithreaded(filename):
    # Load shadow file
    shadow_data = load_shadow_file(filename)
    
    # Use ThreadPoolExecutor to crack passwords concurrently
    with ThreadPoolExecutor(max_workers=8) as executor:
        # Submit cracking tasks for each login
        futures = {
            executor.submit(crack_single_login, login, salt, target_hash): 
            (login, salt, target_hash) 
            for login, salt, target_hash in shadow_data
        }
        
        # Collect results as they complete
        for future in futures:
            try:
                result = future.result()
                if result:
                    print(f"Cracked: {result}")
            except Exception as e:
                print(f"Error processing login: {e}")

def main():
    # Process multiple shadow files
    for i in range(1, 5):
        filename = f"shadow{i}.txt"
        print(f"\nCracking passwords from {filename}:")
        crack_passwords_multithreaded(filename)

if __name__ == "__main__":
    main()


Cracking passwords from shadow1.txt:
Cracked! Login: krajciko, Password: katkA
Cracked! Login: nemecm, Password: milaDa
Cracked: ('krajciko', 'katkA')
Cracked: ('nemecm', 'milaDa')

Cracking passwords from shadow2.txt:
Cracked! Login: kalous, Password: adamKo
Cracked: ('kalous', 'adamKo')

Cracking passwords from shadow3.txt:
Cracked! Login: soukal, Password: adamKo
Cracked: ('soukal', 'adamKo')

Cracking passwords from shadow4.txt:
Cracked! Login: petran, Password: leNka
Cracked! Login: sagan00, Password: adamKo
Cracked: ('petran', 'leNka')
Cracked: ('sagan00', 'adamKo')
